In [101]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [102]:


def get_ohe(train, categ):
    '''
    Функция для ohe_категоризации необходимых столбцов и последующего объединения с количественными столбцами

    На вход: выборка трайн или тест И названия категориальных столбцов
    '''
    temp_df = pd.DataFrame(
        data=ohe.transform(train[[categ]]), columns=ohe.get_feature_names_out()
    )
    data = pd.concat([train.reset_index(drop=True), temp_df], axis=1)
    data = data.drop(columns=categ, axis=1)
    return data



In [103]:
df = pd.read_csv('tg_4_.csv', sep=';', encoding='cp1251', decimal=',')
df = df.dropna()
df

,id,удой,эке,протеин,спо,порода,"Жирность,%"
0,1,5863,14.2,1743.0,0.89,Вис Бик Айдиал,3.58
1,2,5529,12.8,2138.0,0.94,Вис Бик Айдиал,3.54
2,3,5810,14.0,1854.0,0.93,РефлешнСоверинг,103.59
3,4,5895,12.4,2012.0,0.89,РефлешнСоверинг,3.40
5,6,5254,12.7,1806.0,0.89,РефлешнСоверинг,3.13
...,...,...,...,...,...,...,...
147,148,5072,12.3,1815.0,0.84,Вис Бик Айдиал,3.18
148,149,6075,15.2,1864.0,0.94,Вис Бик Айдиал,3.71
149,150,5675,14.1,1814.0,0.94,Вис Бик Айдиал,3.59
150,151,6221,14.7,2156.0,0.93,Вис Бик Айдиал,3.72


In [104]:
interquantile = (df['удой'].quantile(0.75) -
                 df['удой'].quantile(0.25))
df.loc[df['удой'] >= df['удой'].median() + 1.5 * interquantile, 'удой'] = np.nan
df['удой'] = df.groupby('порода')['удой'].transform(
    lambda x: x.fillna(x.median())
)

interquantile = (df['Жирность,%'].quantile(0.75) -
                 df['Жирность,%'].quantile(0.25))
df.loc[df['Жирность,%'] >= df['Жирность,%'].median() + 1.5 * interquantile,
       'Жирность,%'] = np.nan
df['Жирность,%'] = df.groupby('порода')['Жирность,%'].transform(
    lambda x: x.fillna(x.median())
)

In [105]:
df.loc[df['порода'] == 'РефлешнСоверинггггг', 'порода'] = 'РефлешнСоверинг'

In [106]:
df['спо_кат'] = df['спо']\
    .apply(lambda x: 1 if x <= 0.9 else 0)
df['порода_кат'] = df['порода']\
    .apply(lambda x: 1 if x == 'РефлешнСоверинг' else 0)

In [107]:
df.head(5)

,id,удой,эке,протеин,спо,порода,"Жирность,%",спо_кат,порода_кат
0,1,5863.0,14.2,1743.0,0.89,Вис Бик Айдиал,3.580,1,0
1,2,5529.0,12.8,2138.0,0.94,Вис Бик Айдиал,3.540,0,0
2,3,5810.0,14.0,1854.0,0.93,РефлешнСоверинг,3.705,0,1
3,4,5895.0,12.4,2012.0,0.89,РефлешнСоверинг,3.400,1,1
5,6,5254.0,12.7,1806.0,0.89,РефлешнСоверинг,3.130,1,1


In [108]:
x = df[['эке', 'протеин', 'спо_кат', 'порода_кат']]
y = df['удой']
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, random_state=42)

In [109]:
# ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
# ohe.fit(x_train[['порода']])
# ohe.fit(x_test[['порода']])

# x_train = get_ohe(x_train, 'порода')
# x_test = get_ohe(x_test, 'порода')

In [110]:
lin_regr = LinearRegression()
lin_regr.fit(x_train, y_train)
prediction = lin_regr.predict(x_test)

In [111]:
r2 = round(r2_score(y_test, prediction), 2)
mape = round(mean_absolute_percentage_error(y_test, prediction) * 100, 2)

print(r2, mape)

0.8 3.16
